# Computing fare distance time

adapted from here: https://github.com/skyetim/didi-vehicle-repositioning-strategy/blob/master/notebooks/compute_fare_distance_time.ipynb

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from itertools import product
import shapefile
import math

In [2]:
#Read in the data¶
df = pd.read_csv('../data/trip_cleaned.csv')

In [3]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])
df['pickup_time'] = pd.to_datetime(df['pickup_datetime']).dt.time
df['dropoff_time'] = pd.to_datetime(df['dropoff_datetime']).dt.time

df = df.drop(df[df['pickup_datetime'].dt.time == dt.time(0,0)].index)
df = df.drop(df[df['dropoff_datetime'].dt.time == dt.time(0,0)].index)

In [4]:
df.shape

(13873809, 33)

In [5]:
dff = df[['pickup_datetime','dropoff_datetime','fare_amount', 'trip_time', 'trip_distance',
          'pickup_taxizone_id', 'dropoff_taxizone_id','pickup_time','pickup_weekday']]
dff['pickup_datetime'] = pd.to_datetime(dff['pickup_datetime'])
dff['dropoff_datetime'] = pd.to_datetime(dff['dropoff_datetime'])
dff['pickup_datetime_interval'] = dff['pickup_datetime'].dt.round('30min').dt.time
dff['dropoff_datetime_interval'] = dff['dropoff_datetime'].dt.round('30min').dt.time
dff.head()

C:\Users\lizyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\lizyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\lizyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

,pickup_datetime,dropoff_datetime,fare_amount,trip_time,trip_distance,pickup_taxizone_id,dropoff_taxizone_id,pickup_time,pickup_weekday,pickup_datetime_interval,dropoff_datetime_interval
0,2013-06-24 18:55:00,2013-06-24 19:01:00,6.0,6.0,0.97,148.0,79.0,18:55:00,1,19:00:00,19:00:00
1,2013-06-24 19:12:00,2013-06-24 19:30:00,14.5,18.0,3.04,164.0,143.0,19:12:00,1,19:00:00,19:30:00
2,2013-06-24 19:45:00,2013-06-24 20:10:00,17.5,25.0,4.17,142.0,79.0,19:45:00,1,20:00:00,20:00:00
3,2013-06-24 20:17:00,2013-06-24 20:28:00,10.5,11.0,2.35,234.0,141.0,20:17:00,1,20:30:00,20:30:00
4,2013-06-24 20:30:00,2013-06-24 20:35:00,6.0,5.0,1.24,141.0,263.0,20:30:00,1,20:30:00,20:30:00


In [6]:
numintervals = int(24*60/30)

interval_index_table = pd.DataFrame()
interval_index_table['interval'] = [pd.to_datetime(dt.datetime(2013, 6, 1, 0, 0) + \
                                                   dt.timedelta(minutes = i*30)) for i in range(numintervals)]
interval_index_table['interval'] = interval_index_table['interval'].dt.time
interval_index_table['index_30m'] = range(0,48,1)

conversion_30m = dict(zip(interval_index_table['interval'], interval_index_table['index_30m']))

In [7]:
dff['pickup_datetime_index'] = [conversion_30m[t] for t in dff['pickup_datetime_interval']]
dff['dropoff_datetime_index'] = [conversion_30m[t] for t in dff['dropoff_datetime_interval']]
dff['actual_data'] = 1
dff.head()

C:\Users\lizyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\lizyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\lizyu\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

,pickup_datetime,dropoff_datetime,fare_amount,trip_time,trip_distance,pickup_taxizone_id,dropoff_taxizone_id,pickup_time,pickup_weekday,pickup_datetime_interval,dropoff_datetime_interval,pickup_datetime_index,dropoff_datetime_index,actual_data
0,2013-06-24 18:55:00,2013-06-24 19:01:00,6.0,6.0,0.97,148.0,79.0,18:55:00,1,19:00:00,19:00:00,38,38,1
1,2013-06-24 19:12:00,2013-06-24 19:30:00,14.5,18.0,3.04,164.0,143.0,19:12:00,1,19:00:00,19:30:00,38,39,1
2,2013-06-24 19:45:00,2013-06-24 20:10:00,17.5,25.0,4.17,142.0,79.0,19:45:00,1,20:00:00,20:00:00,40,40,1
3,2013-06-24 20:17:00,2013-06-24 20:28:00,10.5,11.0,2.35,234.0,141.0,20:17:00,1,20:30:00,20:30:00,41,41,1
4,2013-06-24 20:30:00,2013-06-24 20:35:00,6.0,5.0,1.24,141.0,263.0,20:30:00,1,20:30:00,20:30:00,41,41,1


In [8]:
def get_shift(time, weekday=True):
    if weekday:
        if (time > dt.time(23, 30)) | (time <= dt.time(11, 30)):
            return 'A'
        else:
            return 'B'
    else: ## weekend
        if (time > dt.time(2, 0)) & (time <= dt.time(14, 0)):
            return 'A'
        else:
            return 'B'
    return None

In [9]:
dff.loc[(dff['pickup_weekday']==1), '_PU_shift'] = dff.pickup_time.apply(lambda x: get_shift(x, weekday=True))
dff.loc[(dff['pickup_weekday']==0), '_PU_shift'] = dff.pickup_time.apply(lambda x: get_shift(x, weekday=False))
dff.head()

C:\Users\lizyu\Anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\lizyu\Anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
C:\Users\lizyu\Anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,pickup_datetime,dropoff_datetime,fare_amount,trip_time,trip_distance,pickup_taxizone_id,dropoff_taxizone_id,pickup_time,pickup_weekday,pickup_datetime_interval,dropoff_datetime_interval,pickup_datetime_index,dropoff_datetime_index,actual_data,_PU_shift
0,2013-06-24 18:55:00,2013-06-24 19:01:00,6.0,6.0,0.97,148.0,79.0,18:55:00,1,19:00:00,19:00:00,38,38,1,B
1,2013-06-24 19:12:00,2013-06-24 19:30:00,14.5,18.0,3.04,164.0,143.0,19:12:00,1,19:00:00,19:30:00,38,39,1,B
2,2013-06-24 19:45:00,2013-06-24 20:10:00,17.5,25.0,4.17,142.0,79.0,19:45:00,1,20:00:00,20:00:00,40,40,1,B
3,2013-06-24 20:17:00,2013-06-24 20:28:00,10.5,11.0,2.35,234.0,141.0,20:17:00,1,20:30:00,20:30:00,41,41,1,B
4,2013-06-24 20:30:00,2013-06-24 20:35:00,6.0,5.0,1.24,141.0,263.0,20:30:00,1,20:30:00,20:30:00,41,41,1,B


# Trip Distance and Time in case of no data

In [10]:
# Credit: https://chih-ling-hsu.github.io/2018/05/14/NYC
# file generated from https://mygeodata.cloud/converter/shp-to-latlong
df_loc = pd.read_csv('../data/taxi_zones/taxi_zones.csv')
df_loc.head(10)

,X,Y,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough
0,-74.176786,40.689516,1,0.116357,0.000782,Newark Airport,1,EWR
1,-73.826126,40.625724,2,0.433470,0.004866,Jamaica Bay,2,Queens
2,-73.849479,40.865888,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx
3,-73.977023,40.724152,4,0.043567,0.000112,Alphabet City,4,Manhattan
4,-74.189930,40.550340,5,0.092146,0.000498,Arden Heights,5,Staten Island
5,-74.067774,40.599062,6,0.150491,0.000606,Arrochar/Fort Wadsworth,6,Staten Island
6,-73.921491,40.761085,7,0.107417,0.000390,Astoria,7,Queens
7,-73.923202,40.778607,8,0.027591,0.000027,Astoria Park,8,Queens
8,-73.788020,40.754411,9,0.099784,0.000338,Auburndale,9,Queens
9,-73.791665,40.678125,10,0.099839,0.000436,Baisley Park,10,Queens


In [11]:
# Credit: https://www.geodatasource.com/resources/tutorials/how-to-calculate-the-distance-between-2-locations-using-python/
def distance(lat1, lon1, lat2, lon2, unit):
    if (lat1 == lat2) and (lon1 == lon2):
        return 0
    else:
        theta = lon1-lon2
        dist = math.sin(math.radians(lat1)) * math.sin(math.radians(lat2)) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(theta))
        dist = math.acos(dist)
        dist = math.degrees(dist)
        miles = dist * 60 * 1.1515;
        unit = unit.upper()

        if unit == 'K':
            return miles * 1.609344
        elif unit == 'N':
            return miles * 0.8684
        else:
            return miles

def distance_len(length, unit):
    miles = length * 60 * 1.1515;
    unit = unit.upper()

    if unit == 'K':
        return miles * 1.609344
    elif unit == 'N':
        return miles * 0.8684
    else:
        return miles

In [12]:
zones = range(1,264,1)
df1 = pd.DataFrame(zones, columns = ['pickup_taxizone_id'])
df2 = pd.DataFrame(zones, columns = ['dropoff_taxizone_id'])

df1['key'] = 0
df2['key'] = 0

dff_loc = df1.merge(df2, on='key', how='outer').drop(columns='key') 

In [13]:
def dist_calc(row):
    if row['pickup_taxizone_id'] == row['dropoff_taxizone_id']:
        # dividing by 6 assuming a 6 sided polygon and then by 2 for average distance between two random points
        return distance_len(df_loc.loc[df_loc['OBJECTID']==row['pickup_taxizone_id']]['Shape_Leng'], 'miles').values[0]/6/2
        
    else:
        # assumption: actual taxi trip travel time averages 20% more than aerial distance
        return distance(df_loc.loc[df_loc['OBJECTID']==row['pickup_taxizone_id']]['X'].values[0], \
                        df_loc.loc[df_loc['OBJECTID']==row['pickup_taxizone_id']]['Y'].values[0], \
                        df_loc.loc[df_loc['OBJECTID']==row['dropoff_taxizone_id']]['X'].values[0], \
                        df_loc.loc[df_loc['OBJECTID']==row['dropoff_taxizone_id']]['Y'].values[0],'miles')

dff_loc['trip_distance'] = dff_loc.apply(lambda row: dist_calc(row), axis=1)

dff_loc.head()

,pickup_taxizone_id,dropoff_taxizone_id,trip_distance
0,1,1,0.669928
1,1,2,24.257527
2,1,3,22.861287
3,1,4,13.817219
4,1,5,2.773706


In [14]:
dff_loc['trip_time'] = dff_loc.trip_distance.apply(lambda x: x/10*60)
# Assumptions: $2.50 base fare + 50 cents per 1/5 of mile driven + 30% assumption for tips and tolls
dff_loc['fare_amount'] = dff_loc.trip_distance.apply(lambda x: (2.50 + x/5*.50)*1.3)
dff_loc
# dff_loc['fare_amount'] = 

,pickup_taxizone_id,dropoff_taxizone_id,trip_distance,trip_time,fare_amount
0,1,1,0.669928,4.019568,3.337091
1,1,2,24.257527,145.545165,6.403479
2,1,3,22.861287,137.167724,6.221967
3,1,4,13.817219,82.903313,5.046238
4,1,5,2.773706,16.642235,3.610582
...,...,...,...,...,...
69164,263,259,6.948938,41.693626,4.153362
69165,263,260,3.336955,20.021729,3.683804
69166,263,261,4.464889,26.789332,3.830436
69167,263,262,0.373463,2.240777,3.298550


In [15]:
dff_loc['actual_data'] = 0
dff_loc.head()
dff_new = pd.concat([dff, dff_loc], ignore_index=True)

# merge(dff, dff_loc, on=['pickup_taxizone_id','dropoff_taxizone_id'], how=’left’)

,pickup_datetime,dropoff_datetime,fare_amount,trip_time,trip_distance,pickup_taxizone_id,dropoff_taxizone_id,pickup_time,pickup_weekday,pickup_datetime_interval,dropoff_datetime_interval,pickup_datetime_index,dropoff_datetime_index,actual_data,_PU_shift
0,2013-06-24 18:55:00,2013-06-24 19:01:00,6.000000,6.000000,0.970000,148.0,79.0,18:55:00,1.0,19:00:00,19:00:00,38.0,38.0,1,B
1,2013-06-24 19:12:00,2013-06-24 19:30:00,14.500000,18.000000,3.040000,164.0,143.0,19:12:00,1.0,19:00:00,19:30:00,38.0,39.0,1,B
2,2013-06-24 19:45:00,2013-06-24 20:10:00,17.500000,25.000000,4.170000,142.0,79.0,19:45:00,1.0,20:00:00,20:00:00,40.0,40.0,1,B
3,2013-06-24 20:17:00,2013-06-24 20:28:00,10.500000,11.000000,2.350000,234.0,141.0,20:17:00,1.0,20:30:00,20:30:00,41.0,41.0,1,B
4,2013-06-24 20:30:00,2013-06-24 20:35:00,6.000000,5.000000,1.240000,141.0,263.0,20:30:00,1.0,20:30:00,20:30:00,41.0,41.0,1,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13942973,0,0,4.153362,41.693626,6.948938,263.0,259.0,0,0.0,0,0,0.0,0.0,0,0
13942974,0,0,3.683804,20.021729,3.336955,263.0,260.0,0,0.0,0,0,0.0,0.0,0,0
13942975,0,0,3.830436,26.789332,4.464889,263.0,261.0,0,0.0,0,0,0.0,0.0,0,0
13942976,0,0,3.298550,2.240777,0.373463,263.0,262.0,0,0.0,0,0,0.0,0.0,0,0


In [19]:
dff_new = dff_new.fillna(0)

In [53]:
dff_new['pickup_datetime_interval'] = dff_new.pickup_datetime_interval.apply(lambda x: x if type(x) == 'datetime.time' else dt.time(0))

# Fare

In [55]:
fare_amount_agg = dff_new.groupby(['pickup_datetime_interval',\
                                   'pickup_taxizone_id','dropoff_taxizone_id','_PU_shift',\
                                   'pickup_weekday','actual_data'], as_index=False)['fare_amount'].agg(['mean', 'count']).reset_index()


In [56]:
fare_amount_agg.shape

(152141, 8)

In [57]:
fare_amount_agg.to_csv('../data/fare_amount_scr_dst_t.csv', index=False)

In [75]:
# fare_amount_agg.loc[(fare_amount_agg['pickup_taxizone_id']==241)&(fare_amount_agg['dropoff_taxizone_id']==174)]

len(fare_amount_agg[(fare_amount_agg["pickup_taxizone_id"] == 241) & 
                                        (fare_amount_agg["dropoff_taxizone_id"] == 174) &
                                        (fare_amount_agg["actual_data"] == 1)])

fare_amount_agg.loc[(fare_amount_agg["pickup_taxizone_id"] == 241) & 
                                        (fare_amount_agg["dropoff_taxizone_id"] == 174) &
                                        (fare_amount_agg["actual_data"] == 0),"mean"].mean()

3.4178784437297955

# Trip Distance

In [76]:
trip_distance_agg = dff_new.groupby(['pickup_taxizone_id', 'dropoff_taxizone_id','_PU_shift','pickup_weekday','actual_data'], as_index=False)['trip_distance'].agg(['mean', 'median', 'count']).reset_index()


In [77]:
trip_distance_agg.shape

(152141, 8)

In [78]:
trip_distance_agg.to_csv('../data/trip_distance_scr_dst.csv', index=False)

# Trip Time

In [84]:
trip_time_agg = dff_new.groupby(['pickup_datetime_interval',\
                             'pickup_taxizone_id', 'dropoff_taxizone_id','_PU_shift','pickup_weekday','actual_data'], \
                            as_index=False)['trip_time'].agg(['mean', 'median', 'count']).reset_index()

In [85]:
trip_time_agg.shape

(152141, 9)

In [86]:
trip_time_agg.to_csv('../data/trip_time_scr_dst_t.csv', index=False)